<h1>Install & Import all necessary components and libraries </h1>
<h3>Before execute this cell you need do this :</h3>
<ul>
    <li>cmd> python -m venv venv</li>
    <li>cmd> venv/Scripts/activate</li>
    <li>(venv)cmd> pip install -r requirements.txt</li>
</ul>

In [41]:
import cv2
import tensorflow as tf
import keras as kr
import numpy as np

from scipy.spatial.distance import cosine as dcos
from scipy.io import loadmat

from keras.models import Sequential, Model
from keras.layers import Flatten, Dropout, Activation, Permute
from keras.layers import Convolution2D, MaxPooling2D
from keras import backend as K
K.set_image_data_format( 'channels_last' )

<h2>Creation of neural network matrix</h2>

In [42]:
def convblock(cdim, nb, bits=3):
    # Declaration of table 
    L = []
    # Browse through all the bits of our image
    for k in range(1,bits+1):
        convname = 'conv'+str(nb)+'_'+str(k)
        L.append(Convolution2D(cdim, kernel_size=(3, 3), padding='same', activation='relu', name=convname))
    L.append(MaxPooling2D((2, 2), strides=(2, 2)))
    return L

In [43]:
def vgg_face_blank():
    withDO = True # no effect during evaluation but usefull for fine-tuning
    if True:
        mdl = Sequential()
        mdl.add( Permute((1,2,3), input_shape=(224,224,3)) )
        for l in convblock(64, 1, bits=2):
            mdl.add(l)
        for l in convblock(128, 2, bits=2):
            mdl.add(l)        
        for l in convblock(256, 3, bits=3):
            mdl.add(l)            
        for l in convblock(512, 4, bits=3):
            mdl.add(l)            
        for l in convblock(512, 5, bits=3):
            mdl.add(l)  
                  
        mdl.add( Convolution2D(4096, kernel_size=(7, 7), activation='relu', name='fc6') )
        if withDO:
            mdl.add( Dropout(0.5) )

        mdl.add( Convolution2D(4096, kernel_size=(1, 1), activation='relu', name='fc7') )
        if withDO:
            mdl.add( Dropout(0.5) )

        mdl.add( Convolution2D(2622, kernel_size=(1, 1), activation='relu', name='fc8') )
        mdl.add( Flatten() )
        mdl.add( Activation('softmax') )
        return mdl
    
    else:
        raise ValueError('not implemented')

<h2>Importation of neural weight & bias</h2>

In [44]:
def copy_mat_to_keras(kmodel):
    kerasnames = [lr.name for lr in kmodel.layers]
    prmt = (0,1,2,3)
 
    for i in range(l.shape[1]):
        matname = l[0,i][0,0].name[0]
        if matname in kerasnames:
            kindex = kerasnames.index(matname)
            l_weights = l[0,i][0,0].weights[0,0]
            l_bias = l[0,i][0,0].weights[0,1]
            f_l_weights = l_weights.transpose(prmt)
            assert (f_l_weights.shape == kmodel.layers[kindex].get_weights()[0].shape)
            assert (l_bias.shape[1] == 1)
            assert (l_bias[:,0].shape == kmodel.layers[kindex].get_weights()[1].shape)
            assert (len(kmodel.layers[kindex].get_weights()) == 2)
            kmodel.layers[kindex].set_weights([f_l_weights, l_bias[:,0]])

<h2>Functions execution</h2> 

In [45]:
# CNN model initialization
facemodel = vgg_face_blank()

# Load the pretrained weights into the model
data = loadmat('vgg-face.mat', matlab_compatible=False, struct_as_record=False)
l = data['layers']
description = data['meta'][0,0].classes[0,0].description

# Importation of neural weight & bias with facemodel 
copy_mat_to_keras(facemodel)

# Final model that can get inputs and generate a prediction as an output
featuremodel = Model( inputs = facemodel.layers[0].input, outputs = facemodel.layers[-2].output )

In [46]:

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
image = load_img("saved_img-final.jpg")
vector_image = img_to_array(image)
# featuremodel.predict(vector_image)[0,:]

TypeError: float() argument must be a string or a number, not '_io.TextIOWrapper'